# Final Assignment Project Capstone

### Author: Kobi Zohar

### Problem Desciption:
I'd like to buy house in the UK, therefire i'd like o emine / determine if house price corelative to number of venues near the property (upto 500m). That's will make my chiose easier.


### Data Gathering:
i found at the internet average sale price for houses in the UK, associated with different region names. in addition i used geocoder in order to extract the lat and long for each location. and find number of nearby venues with foursquare API. i arrenge the data that way, i can analyze if i can cluster / predict average sale price by the number of venues nearby the location.

### Importing relevant libreries

In [ ]:
from bs4 import BeautifulSoup
import requests
from pandas.io.html import read_html
import pandas as pd
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


In [2]:
address = 'Aberdeenshire'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


57.166667 -2.666667


In [3]:
file=open(r'C:\Users\zoharko\Documents\Python Scripts\Data Scientist Course\Capstone Project Notebook\UK_Prices.csv')
df=pd.read_csv(file)
df.head()

,Date,RegionName,AreaCode,AveragePrice
0,01/01/2018,Aberdeenshire,S12000034,191808.8987
1,01/02/2018,Aberdeenshire,S12000034,191803.0667
2,01/03/2018,Aberdeenshire,S12000034,191213.1247
3,01/04/2018,Aberdeenshire,S12000034,188293.4953
4,01/05/2018,Aberdeenshire,S12000034,188197.5303


In [117]:
df1=df.groupby('RegionName').mean()
df2 = df1.sample(n=50)
df2.head()

,AveragePrice
RegionName,
Wokingham,413187.056400
Moray,149815.371714
Staffordshire Moorlands,170485.826600
East Renfrewshire,222904.582429
Cambridgeshire,291641.101686


In [118]:
import time
add=[]
for address in df2.index:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    time.sleep(1)    
    add.append([address,location.latitude,location.longitude])
    
    

In [50]:
df_add=pd.DataFrame(add)
df_add.rename(columns={0:'RegionName',1:'latitude',2:'longitude'}, inplace=True)
df_add.head()

,RegionName,latitude,longitude
0,West Devon,50.651997,-4.074311
1,Greater Manchester,53.506525,-2.337440
2,West Somerset,51.118735,-3.503575
3,Waltham Forest,51.556999,-0.005835
4,Birmingham,52.477540,-1.894053


In [53]:
df_merged = pd.merge(df2, df_add, on='RegionName')
df_merged.head()

,RegionName,AveragePrice,latitude,longitude
0,West Devon,245078.507200,50.651997,-4.074311
1,Greater Manchester,164241.577271,53.506525,-2.337440
2,West Somerset,225104.956957,51.118735,-3.503575
3,Waltham Forest,438979.202100,51.556999,-0.005835
4,Birmingham,180427.574286,52.477540,-1.894053


In [54]:
CLIENT_ID = 'ZDIFMWFBVE5QV4NTOWZ2YWSOKZF2K5MBIKVW1QXKP3DOGCPS' # your Foursquare ID
CLIENT_SECRET = 'AQVGWP3UHSLFIKIYFRARLCWH33V5XASJZ1335EJCZZI1U0NW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZDIFMWFBVE5QV4NTOWZ2YWSOKZF2K5MBIKVW1QXKP3DOGCPS
CLIENT_SECRET:AQVGWP3UHSLFIKIYFRARLCWH33V5XASJZ1335EJCZZI1U0NW


In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
LIMIT=500
uk_venues = getNearbyVenues(names=df_merged['RegionName'],
                                   latitudes=df_merged['latitude'],
                                   longitudes=df_merged['longitude'])

West Devon
Greater Manchester
West Somerset
Waltham Forest
Birmingham
East Dorset
South Lanarkshire
Waveney
South Derbyshire
Cambridgeshire
Bath and North East Somerset
Chorley
Merthyr Tydfil
Ashfield
Rotherham
City of Aberdeen
Perth and Kinross
Swindon
City of Peterborough
East Lothian
Fermanagh and Omagh
Forest Heath
Bromley
North Somerset
Broxtowe
Torridge
Mendip
Slough
North Ayrshire
Surrey Heath
Buckinghamshire
West Lothian
Gateshead
Weymouth and Portland
Southampton
Pembrokeshire
Lambeth
North East Derbyshire
Bromsgrove
Kent
Redbridge
Midlothian
Blackpool
Rossendale
Daventry
Sunderland
West Lindsey
Hertsmere
Forest of Dean
Enfield


In [75]:
uk_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,West Devon,50.651997,-4.074311,Brat Tor,50.651501,-4.067323,Trail
1,Greater Manchester,53.506525,-2.337440,The White Swan,53.505701,-2.341182,Bar
2,Greater Manchester,53.506525,-2.337440,Top Wok Express,53.505085,-2.333285,Chinese Restaurant
3,Greater Manchester,53.506525,-2.337440,Agra,53.507923,-2.332408,Indian Restaurant
4,Greater Manchester,53.506525,-2.337440,Salford Precinct,53.503944,-2.341384,Shopping Mall


In [111]:
df_venue_count=uk_venues.groupby(['Neighborhood']).count()
df_venue_count.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'],axis=1,inplace=True)
df_venue_count.head()

,Venue
Neighborhood,
Ashfield,4
Bath and North East Somerset,3
Birmingham,91
Blackpool,69
Bromley,45


In [112]:
final_df= pd.merge(df_merged, df_venue_count, left_on='RegionName', right_on=df_venue_count.index, how='left')
final_df['Venue'] = final_df['Venue'].replace('nan', np.nan).fillna(0)


In [114]:
final_df.head()

,RegionName,AveragePrice,latitude,longitude,Venue
0,West Devon,245078.507200,50.651997,-4.074311,1.0
1,Greater Manchester,164241.577271,53.506525,-2.337440,6.0
2,West Somerset,225104.956957,51.118735,-3.503575,0.0
3,Waltham Forest,438979.202100,51.556999,-0.005835,38.0
4,Birmingham,180427.574286,52.477540,-1.894053,91.0


,AveragePrice,latitude,longitude,Venue
AveragePrice,1.000000,-0.533380,-0.072273,0.184314
latitude,-0.533380,1.000000,0.569840,-0.058595
longitude,-0.072273,0.569840,1.000000,0.082818
Venue,0.184314,-0.058595,0.082818,1.000000
